**Sorry if this notebook looks kinda messy and also sorry for my bad english**
<br>~
<br>Sources/Credits :
<br>FrameBot by [Boidushya](https://github.com/Boidushya)
<br>Colab Notebook by [izu14](https://github.com/izu14)
<br>~
<br>Before proceeding, please read this [README.md](https://github.com/izu14/FrameBot-Colab/blob/master/README.md)

****

**TEST YOUR GOOGLE COLAB INTERNET SPEED**
<br>MAKE SURE YOU GET A GOOD RESULT ESPECIALLY THE UPLOAD SPEED
<br>SINCE THIS WILL AFFECT HOW THE BOT WORKS

In [ ]:
#@markdown ### ← Run this to test google colab internet speed
from IPython.display import clear_output
!sudo apt-get install gnupg1 apt-transport-https dirmngr
!export INSTALL_KEY=379CE192D401AB61
!sudo apt-key adv --keyserver keyserver.ubuntu.com --recv-keys $INSTALL_KEY
!echo "deb https://ookla.bintray.com/debian generic main" | sudo tee  /etc/apt/sources.list.d/speedtest.list
!sudo apt-get update
clear_output()
!sudo apt-get install speedtest-cli
clear_output()
!speedtest --simple

****

**[1] CLONE FRAMEBOT REPO AND INSTALL REQUIRED PACKAGES**

In [ ]:
#@markdown ### ← Run this
from IPython.display import clear_output
!git clone https://github.com/izu14/FrameBot
clear_output()
%mkdir /content/FrameBot/assets/video
%cd /content/FrameBot
clear_output()
!sudo apt update
clear_output()
!sudo apt install ffmpeg
!sudo pip3 install -r requirements.txt
clear_output()
!sudo pip3 install -q youtube-dl
clear_output()
print(f"Done")

**[2] CHOOSE VIDEO SOURCE(Only Support Single Video Input)**

In [ ]:
# @markdown ###← YouTube or any [ytdl supported sites](https://gitlab.com/ytdl-org/youtube-dl/-/raw/master/docs/supportedsites.md)
from IPython.display import clear_output
%rm /content/FrameBot/assets/video/*.mp4 
clear_output()
Link = "putvideolinkhere" #@param{type:"string"}
!youtube-dl -f 'bestvideo[ext=mp4]' --output "/content/FrameBot/assets/video/%(title)s.%(ext)s" $Link
clear_output()
%cd /content/FrameBot/assets/video
clear_output()
!for filename in *.mp4; do echo 'Downloaded '"$filename"; done
!for filename in /content/FrameBot/assets/video/*.mp4; do mv $filename /content/FrameBot/assets/video/input.mp4; done
!find /content/FrameBot/assets/video -mindepth 1 ! -regex '^/content/FrameBot/assets/video/input.mp4\(/.*\)?' -delete

In [ ]:
#@markdown ###← Local file
from IPython.display import clear_output
import google.colab.files
%rm /content/FrameBot/assets/video/*.mp4
clear_output()
%cd /content/FrameBot/assets/video
clear_output()
uploaded = google.colab.files.upload()
!for filename in /content/FrameBot/assets/video/*.mp4; do echo 'Uploaded'"$filename"; done
!for filename in /content/FrameBot/assets/video/*.mp4; do mv $filename /content/FrameBot/assets/video/input.mp4; done
!find /content/FrameBot/assets/video -mindepth 1 ! -regex '^/content/FrameBot/assets/video/input.mp4\(/.*\)?' -delete
#@markdown > For now, please only use .mp4 file

**[OPTIONAL] CUT/TRIM THE VIDEO**

In [ ]:
#@markdown ###← Press this after you fill the form
from IPython.display import clear_output
START = "00:00:00" #@param{type:"string"}
END = "00:00:00" #@param{type:"string"}
!find /content/FrameBot/assets/video -mindepth 1 ! -regex '^/content/FrameBot/assets/video/input.mp4\(/.*\)?' -delete
%cd /content/FrameBot/assets/video
!ffmpeg -i input.mp4 -vcodec copy -acodec copy -ss $START -t $END croppedinput.mp4
!find /content/FrameBot/assets/video -mindepth 1 ! -regex '^/content/FrameBot/assets/video/croppedinput.mp4\(/.*\)?' -delete
clear_output()
print(f"Done")
#@markdown > Format Used HH:MM:SS
#@markdown > <br>Example : 01:02:03 means 01 Hour 02 Minutes 03 Seconds

**[3] RUNNING THE BOT**

In [ ]:
#This code was written by Boidushya Bhattacharya and Gustav Wallström (github/sudoxd) on Monday, 26 November 2019 at 20:27 p.m.
#Reddit: https://reddit.com/u/Boidushya
#Facebook: https://facebook.com/soumyadipta.despacito
 
#@markdown ###← Press this after you fill all the required form
from IPython.display import clear_output
import cv2
import os
import math
import facebook
import functools
import schedule
import time
import fnmatch
import sys
 
#@markdown ###<br> ↓ Configure your facebook page access token
Token = "putyourtokenhere" #@param{type:"string"}
%store Token > /content/FrameBot/assets/token.txt
clear_output()
#@markdown > How to get your own token : <br>https://github.com/Boidushya/FrameBot/blob/master/generateToken.md</p>
 
#@markdown ###<br> ↓ Configure your post caption
Title = "aiueo" #@param{type:"string"}
#@markdown > aiueo will give a result like : <br>aiueo - Frame 1 out of 2
 
#@markdown ###<br> ↓ Change the FPS for your video here
FPS=1#@param{type:"number"}
#@markdown > The more the fps, the shorter interval between each frames,the more number of frames, the longer it'll take to post the whole frame
#@markdown > <br>Recommended FPS : 1 or 2

#@markdown ###<br> ↓ Interval/delay(in Seconds) for each post
Interval=15#@param{type:"number"}
#@markdown > If you put 15 then the bot will run to post the frame every 15 seconds
#@markdown > <br>this interval will be affected by colab connection quality, video file too, so it's not 100% accurate

##@markdown ###<br> ↓ Extract Frame Files into Colab Directory
#Extract = "Yes" #@param ["Yes", "No"] {allow-input: true}
##@markdown > The files will be stored to /content/FrameBot/assets/frames
 
def catch_exceptions(cancel_on_failure=False):
    def catch_exceptions_decorator(job_func):
        @functools.wraps(job_func)
        def wrapper(*args, **kwargs):
            try:
                return job_func(*args, **kwargs)
            except:
                import traceback
                print(traceback.format_exc())
                if cancel_on_failure:
                    return schedule.CancelJob
        return wrapper
    return catch_exceptions_decorator
 
def extractFrames():
    file = os.listdir('/content/FrameBot/assets/video')[0]
    videoFile = f"/content/FrameBot/assets/video/{file}"
    if not os.path.exists('/content/FrameBot/assets/frames'):
        os.mkdir('/content/FrameBot/assets/frames')
    if os.path.exists("/content/FrameBot/assets/frames/*.jpg"):
        os.remove("/content/FrameBot/assets/frames/*.jpg")
    vidcap = cv2.VideoCapture(videoFile)
    success,image = vidcap.read()
    fpsx = vidcap.get(cv2.CAP_PROP_FPS)
    required_fps = FPS
    multiplier = round(fpsx/required_fps)
    x=0
 
    while success:
        frameId = int(round(vidcap.get(1)))
        success, image = vidcap.read()
 
        if frameId % multiplier == 0:
            x+=1
            cv2.imwrite(f"/content/FrameBot/assets/frames/frame{int(x):06d}.jpg", image)
    vidcap.release()
    
@catch_exceptions()
def post():
    dir = os.listdir("/content/FrameBot/assets/frames")
    dir.sort(key = lambda t : int(t[5:-4])) #forgot to sort the files before, pls forgive me lol
    with open("/content/FrameBot/assets/retain","a+") as f:
        f.seek(0)
        filled = f.read(1)
        if not filled:
            totalFrames = str(len(dir))
            f.write(totalFrames)
        else:
            f.seek(0)
            totalFrames = str(f.readline())
 
    currentFrame = f'/content/FrameBot/assets/frames/{dir[0]}'
    currentFrameNumber = str(int(dir[0][5:-4]))
    msg = f"{Title} - Frame {currentFrameNumber} out of {str(totalFrames)}"
    with open('/content/FrameBot/assets/token.txt','r') as token:
        accesstoken = token.readline()
    graph = facebook.GraphAPI(accesstoken)
    post_id = graph.put_photo(image=open(currentFrame, 'rb'),message = msg)['post_id']
    print(f"Submitted post with title \"{msg}\" successfully!")
    os.remove(currentFrame)
 
if __name__ == '__main__':
    token = open('/content/FrameBot/assets/token.txt', 'r')
    if token.readline() == "putyourtokenhere":
        print("please replace putyourtokenhere with your own token")
        sys.exit("error no token")
    ans = "y"
    if 'y' in ans.lower():
        if os.path.exists("/content/FrameBot/assets/retain"):
            os.remove("/content/FrameBot/assets/retain")
        extractFrames()
    else:
        pass
    schedule.every(Interval).seconds.do(post).run()
 
    while 1:
        schedule.run_pending()
        time.sleep(1)

****

##### AFTER THE LAST FRAME POSTED, YOU MUST PRESS THE STOP BUTTON(IT WAS RUN/PLAY BUTTON PREVIOUSLY) ON THE LEFT TO STOP THIS CODE OR RIGHT CLICK ON IT THEN CLICK INTERRUPT EXECUTION, ELSE, IT'LL ALWAYS THROW/SHOW AN ERROR.

##### <br>IF YOU WANT TO RUN THE BOT AGAIN, IT'S RECOMMENDED TO FACTORY RESET YOUR COLAB RUNTIME FIRST THEN RUN THE CODE FROM THE BEGINNING AGAIN, ELSE (PROBABLY)IT'LL MAKE THE INTERVAL FEATURE NOT WORK PROPERLY.